# MNIST-CNN

In [1]:
import tensorflow as tf

In [2]:
tf.compat.v1.disable_eager_execution()

In [3]:
mnist = tf.keras.datasets.mnist

In [4]:
# Init Weights

def init_weights(shape):
    init_random_dist = tf.compat.v1.truncated_normal(shape = shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [5]:
# Initating the bias values

def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)
    

In [6]:
#creating a convinence function to create a Conv2D

def conv2d(x,W):
    # x --> [batch,W,H,Channels]
    # W --> [filter H, filter W, Channels IN, Channels OUT]
    
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [7]:
# POOLING


In [8]:
def max_pool_2by2(x):
    # x --> [batch,h,w,c] 
    return tf.nn.max_pool(x,ksize=[1,2,2,1],padding= 'SAME',strides=[1,2,2,1])




In [9]:
#CONVOLUTION LAYER
def convolutional_layer(input_x,shape):
    W = init_weights(shape=shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+ b)
       

In [11]:
#NORMAL (FULLY_CONNECTED)

def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W)+ b
    




In [12]:
#PLACEHOLDERS
x = tf.compat.v1.placeholder(tf.float32,shape=[None,784])

y_true = tf.compat.v1.placeholder(tf.float32,shape=[None,10])

In [13]:
# Layers 
x_image = tf.reshape(x,shape=[-1,28,28,1])

In [14]:
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
#now let's add its pooling layer
convo_1_pooling = max_pool_2by2(convo_1)

In [15]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
#now let;s add it;s pooling layer
convo_2_pooling = max_pool_2by2(convo_2)

In [16]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
#we are going to use rectifier linear unit
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [17]:
#Drop out 
hold_prob = tf.compat.v1.placeholder(tf.float32)
full_one_dropout = tf.compat.v1.nn.dropout(full_layer_one,keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
y_pred= normal_full_layer(full_one_dropout,10)

In [19]:
#Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [20]:
#OPTIMIZER
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
#adding a global initalizer
init = tf.compat.v1.global_variables_initializer()

In [22]:
sess = tf.compat.v1.Session()

In [23]:
steps = 1000

sess.run(init)

for i in range(steps):
    batch_x , batch_y = mnist.tr